In [1]:
import pandas as pd
from tqdm.autonotebook import tqdm
import re
from sklearn.metrics.pairwise import cosine_similarity

/tmp/ipykernel_202033/286499899.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_csv("../parsing/paragraphs.csv.zip", usecols=["cleaned_text"])
df = df.dropna()
df.head()

/tmp/ipykernel_58135/177660427.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../parsing/paragraphs.csv.zip", usecols=["cleaned_text"])


,cleaned_text
0,ммк
1,социальный отчёт ммк за год магнитогорский мет...
3,содержание
4,обращение председатель совет директор параметр...
5,ммк


In [3]:
df.shape

(2309485, 1)

In [4]:
df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
df = df.reset_index(drop=True)
df.shape

(1963163, 1)

In [8]:
sheets_dict = {}
base_dict = {}
re_dict = {}
word_list = []

excel = pd.ExcelFile("./esg_topics.xlsx")
for sheet_name in excel.sheet_names:
    sheet_df = pd.read_excel(excel, sheet_name=sheet_name, header=None)
    word_list.extend([{"topic": sheet_name, "word": row[0], "type": row[1]} for _, row in sheet_df.iterrows()])
    sheets_dict[sheet_name] = {row[0]: 1 for _, row in sheet_df.iterrows()}
    base_dict[sheet_name] = {row[0]: 0 for _, row in sheet_df.iterrows()}
    # In this updated pattern, (?!\w) is a negative lookahead assertion that ensures "le" is not followed by a word character. This will correctly match "simple example" but will not match the standalone "le" in "easy" or "alone." Now, the regex should produce the desired output with only "simple example" being matched.
    re_dict[sheet_name] = re.compile("(" + "(?!\w)|".join(sheet_df[0].tolist()) + "(?!\w))")

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [14]:
pd.DataFrame(word_list).to_csv("e_topic.csv.zip", index=False)

In [3]:
text = "социальный отчёт полностью охватывать социальный политика оао ммк дальнейший расширение возможно часть увеличение показатель деятельность случай выявление заинтересованность пользователь информация общество"

for key, regex in re_dict.items():
    print(key, regex.findall(text))

Экология []
Климат ['политика']
Энергия []
Воздух []
Вода []
Отходы []
Биоразнообразие []
Социально-культурное воздействи []


In [7]:
import copy

topics = []
for text in tqdm(df["cleaned_text"].tolist()):
    paragraph_dict = copy.deepcopy(base_dict)
    found_words = False
    for key, regex in re_dict.items():
        words = regex.findall(text)
        for word in words:
            paragraph_dict[key][word] = 1
            found_words = True
    res = {}
    if not found_words:
        res = {key: 0 for key in sheets_dict}
        topics.append(res | {"max_score": None, "max_topic": None})
        continue
    for key, vector in sheets_dict.items():
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
    
    topics.append(res | {"max_score": max(res.values()), "max_topic": max(res, key=lambda k: res[k])})

  0%|          | 0/1963163 [00:00<?, ?it/s]

In [8]:
df_topic = pd.DataFrame(topics) 
df_topic["max_topic"].value_counts()

Экология           285751
Климат             208173
Энергия             78585
Отходы              59408
Вода                41847
Биоразнообразие      8287
Воздух               6894
Name: max_topic, dtype: int64

In [10]:
df_topic.to_csv("./topics.csv.zip", index=False)

In [ ]:
df = pd.read_csv("../parsing/paragraphs.csv.zip",0)
df = df.dropna()